# 라이브러리 import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

path = 'C:\\Users\\wldbs\\Desktop\\kart\\'

## 컬럼 설명
- cnt_match matchid 기준 건수  
- cnt_account 사용자 기준 건수 (retire된 유저 포함)  
- no_retire_SUM_time : 매치 타임을 전부 합한 값 (retire된 유저 포함)   
- no_retire_SUM_time.1 : no_retire_SUM_time에서 retire된 유저를 제외한 평균값  
- cnt_retire_player : 리타이어된 유저  
- cnt_player : 전체 유저  
- percent_retire : 리타이어된 유저 비율 

# KPI 데이터로 분포 확인

In [ ]:
kpi = pd.read_csv(path + "00-single_speed_10031010-KPI.csv")
kpi.head(3)

In [ ]:
# 하루에 한 번도 플레이 하지 않은 맵 (4일 동안 최소 4번, n일동안 최소 n번) 은 삭제하기로 함

cnt_df = kpi.loc[kpi.cnt_match >= 4]

In [ ]:
# 이상치 확인 함수
def detect_num(df):
    for i in df.select_dtypes(exclude='object').columns:
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        print( i, df[i][ (df[i]>Q3 + IQR*1.5) | (df[i]<Q1 - IQR*1.5) ].count() )
        print( i, 'Q1 이하: ', df[i][ (df[i]<Q1 - IQR*1.5) ].count() )
        print( i, 'Q3 이상: ', df[i][ (df[i]>Q3 + IQR*1.5) ].count() )
        print( i, '이상치의 전체 비율: ', round(df[i][ (df[i]>Q3 + IQR*1.5) | (df[i]<Q1 - IQR*1.5) ].count() / df[i].count(), 5) * 100 )
        print("="*36)

In [ ]:
# 전체 매치수

cnt_df.cnt_match.sum()

In [ ]:
# 사용률 백분위 컬럼 생성

cnt_df['percentage'] = round(cnt_df.cnt_match/13443*100, 3)

In [ ]:
# 로그 씌우기 전 cnt값 사분위수 확인하기

Q1 = cnt_df['cnt_match'].quantile(0.25)
Q2 = cnt_df['cnt_match'].quantile(0.5)
Q3 = cnt_df['cnt_match'].quantile(0.75)

print('Q1 : ', Q1)
print('Q2 : ', Q2)
print('Q3 : ', Q3)

print('median : ', cnt_df.cnt_match.median())

In [ ]:
# cnt_match에 log 씌워보기

cnt_df['log_cnt'] = np.log1p(cnt_df['cnt_match'])
test_df = cnt_df[['map_name', 'cnt_match', 'log_cnt']]

In [ ]:
# 이상치 갯수 확인

detect_num(test_df)

In [ ]:
# LOG
# Q1 - Q2
# Q3 이상

# 로그 씌운 cnt값 사분위수 확인하기

log_Q1 = test_df['log_cnt'].quantile(0.25)
log_Q2 = test_df['log_cnt'].quantile(0.5)
log_Q3 = test_df['log_cnt'].quantile(0.75)

print('Q1 : ', log_Q1)
print('Q2 : ', log_Q2)
print('Q3 : ', log_Q3)

print('median : ', cnt_df.log_cnt.median())

## 시각화로 데이터 분포 확인하기  
- log_cnt에 대한 boxplot 출력
- 이상치 분포 확인 필요


In [ ]:
# 이상치 3개

test_df[['log_cnt']].boxplot(figsize = (15, 10))

In [ ]:
# cnt_match에 대한 boxplot 출력
# log 씌우지 않은 데이터로 확인하기 위해 출력해봄

test_df[['cnt_match']].boxplot(figsize = (15, 10))

In [ ]:
# LOG
# Q1 - Q2 테이블로 만들기
# 8 - 19회, 53건

LOG_Q1Q2_df = test_df.loc[(test_df.log_cnt >= log_Q1) & (test_df.log_cnt <= log_Q2)]
# LOG_Q1Q2_df

In [ ]:
# LOG
# Q3 이상 테이블로 만들기
# 57이상, 48건

LOG_upper_Q3_df = test_df.loc[test_df.log_cnt >= log_Q3]
# LOG_upper_Q3_df

In [ ]:
# 저장하기

LOG_upper_Q3_df.to_csv(path + 'upper_map.csv', index=False, encoding = 'utf-8')
LOG_Q1Q2_df.to_csv(path + 'lower_map.csv', index=False, encoding = 'utf-8')

### 혜민님 요청 데이터

In [ ]:
# 변수명 변경 (코드 간소화를 위함)

upper_map = LOG_upper_Q3_df
lower_map = LOG_Q1Q2_df

In [ ]:
# 요청 데이터 1 (Q2 ~ Q3를 포함하지 않는 사이 모든 데이터 출력)

request1 = test_df.loc[(test_df.log_cnt > log_Q2) & (test_df.log_cnt < log_Q3)]
# request1

In [ ]:
# 요청 데이터 2 - 1 (Q2를 포함하여 Q2 상위 25건)

request2_1 = test_df.loc[test_df.log_cnt >= log_Q2]
request2_1 = request2_1.head(25)

In [ ]:
# 요청 데이터 2 - 2 (Q2를 포함하여 Q2 하위 25건)

request2_2 = test_df.loc[test_df.log_cnt <= log_Q2]
request2_2=request2_1.head(25)

In [ ]:
# 요청 데이터 2-1, 2-2 합침

request2 = pd.concat([request2_1, request2_1], axis = 0)

In [ ]:
# # 저장

# request1.to_csv(path + 'request1.csv', index=False, encoding = 'utf-8')
# request2.to_csv(path + 'request2.csv', index=False, encoding = 'utf-8')